# General nuclear segmentation training notebook

This notebook trains a model to perform nuclear segmentation of fluorescent microscopy images.

## Part 1: Import relevant python packages

This section imports python packages that are used throughout the notebook and defines parameters that can be used with papermill

In [1]:
import os
import errno

import numpy as np

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
        
import deepcell

In [2]:
# Define parameters that can be set with papermill
backbone = 'resnet50'
n_epochs = 16
date = '04092020'
filename = 'general_nuclear_train_large.npz'
dataset_name = 'general_nuclear_train_large'
dataset_size = 82800
dataset_split_seed = 0
model_type = 'pixelwise'
batch_size = 4 if model_type == 'retinamask' else 16

In [3]:
# Parameters
n_epochs = 16
batch_size = 16
date = "04162020"
filename = "nuclear_1_128_mobilenetv2_watershed"
dataset_split_seed = 1
dataset_size = 128
backbone = "mobilenetv2"
model_type = "watershed"


## Part 2: Load training data and create data generators

Data generators augment data and feed it into the model training process. Here, we define the generators that will be used to train our segmentation model.

In [4]:
from deepcell.utils.data_utils import get_data
from deepcell import image_generators
from deepcell.utils import train_utils
from sklearn.model_selection import train_test_split

# Load dataset based 
dataset_direc = '/data/training_data/'
training_file = 'general_nuclear_train_large.npz'
test_file = 'general_nuclear_test_large.npz'

# Load training data
training_data = np.load(os.path.join(dataset_direc,training_file))
X = training_data['X']
y = training_data['y']

# Shuffle training data
index = np.arange(X.shape[0])
np.random.seed(dataset_split_seed)
index = np.random.permutation(index)
index = index[0:dataset_size]
X = X[index]
y = y[index]

# Split training data
X_train, X_val, y_train, y_val = train_test_split(X, y, 
                                                random_state=dataset_split_seed, 
                                                shuffle=False, 
                                                test_size=0.2)
train_dict = {'X':X_train, 'y':y_train}
val_dict = {'X':X_val, 'y':y_val}
test_dict = np.load(os.path.join(dataset_direc, test_file))

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(102, 128, 128, 1) (102, 128, 128, 1) (26, 128, 128, 1) (26, 128, 128, 1)


In [5]:
from deepcell.utils.retinanet_anchor_utils import generate_anchor_params
from tensorflow.python.keras import backend as K

# Get anchor settings for RetinaMask models
pyramid_levels = ['P3', 'P4']
anchor_size_dict = {'P3':16, 'P4':32}
anchor_params = generate_anchor_params(pyramid_levels, anchor_size_dict)

# Data augmentation parameters
generator_kwargs = {}
generator_kwargs['rotation_range'] = 180
generator_kwargs['shear_range'] = 0
generator_kwargs['zoom_range'] = 0.25
generator_kwargs['horizontal_flip'] = True
generator_kwargs['vertical_flip'] = True

generator_val_kwargs = {}
generator_val_kwargs['rotation_range'] = 0
generator_val_kwargs['shear_range'] = 0
generator_val_kwargs['zoom_range'] = 0
generator_val_kwargs['horizontal_flip'] = False
generator_val_kwargs['vertical_flip'] = False

# Minimum number of objects in an image
min_objects = 3 if model_type == 'retinamask' else 0

# Random seed
seed=808

if model_type == 'watershed':
    datagen = image_generators.SemanticGenerator(**generator_kwargs)
    datagen_val = image_generators.SemanticGenerator(**generator_val_kwargs)
    
    train_data = datagen.flow(
        train_dict,
        seed=seed,
        transforms=['centroid', 'watershed-cont', 'fgbg'],
        transforms_kwargs={'watershed-cont': {'erosion_width': 0}},
        min_objects=min_objects,
        batch_size=batch_size)

    val_data = datagen_val.flow(
        val_dict,
        seed=seed,
        transforms=['centroid', 'watershed-cont', 'fgbg'],
        transforms_kwargs={'watershed-cont': {'erosion_width': 0}},
        min_objects=min_objects,
        batch_size=batch_size)
    
elif model_type == 'pixelwise':
    datagen = image_generators.SemanticGenerator(**generator_kwargs)
    datagen_val = image_generators.SemanticGenerator(**generator_val_kwargs)
    
    train_data = datagen.flow(
        train_dict,
        seed=seed,
        transforms=['pixelwise'],
        transforms_kwargs={},
        min_objects=min_objects,
        batch_size=batch_size)

    val_data = datagen_val.flow(
        val_dict,
        seed=seed,
        transforms=['pixelwise'],
        transforms_kwargs={},
        min_objects=min_objects,
        batch_size=batch_size)

elif model_type == 'retinamask':
    datagen = image_generators.RetinaNetGenerator(**generator_kwargs)
    datagen_val = image_generators.RetinaNetGenerator(**generator_val_kwargs)
    
    train_data = datagen.flow(
        train_dict=train_dict,
        seed=seed,
        transforms=[],
        transforms_kwargs={},
        min_objects=min_objects,
        batch_size=batch_size,
        anchor_params=anchor_params,
        pyramid_levels=pyramid_levels,
        include_masks=True)
    
    val_data = datagen_val.flow(
        train_dict=val_dict,
        seed=seed,
        transforms=[],
        transforms_kwargs={},
        min_objects=min_objects,
        batch_size=batch_size,
        anchor_params=anchor_params,
        pyramid_levels=pyramid_levels,
        include_masks=True)
    

## Part 3: Define model

Here we define a PanopticNet to perform the image segmentation. This model will predict the inner distance and outer distance transform (as done in ), as well as the foreground-background transform.

In [6]:
from deepcell.model_zoo.panopticnet import PanopticNet
from deepcell.model_zoo.maskrcnn import RetinaMask
from deepcell import losses
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.losses import MSE

# Define optimizer
optimizer = Adam(lr=1e-4, clipnorm=0.001)
    
if model_type == 'watershed':
    model = PanopticNet(backbone,
                       input_shape=train_data.x.shape[1:],
                       norm_method='whole_image',
                       num_semantic_heads=3,
                       num_semantic_classes=[1,1,2],
                       location=True,
                       include_top=True)

    # Define loss
    loss_layers = ['semantic_0', 'semantic_1', 'semantic_2']
    loss = {}

    for layer_name in loss_layers:
        n_classes = model.get_layer(layer_name).output_shape[-1]
        if n_classes > 1:
            def loss_function(y_true, y_pred):
                return 0.01 * losses.weighted_categorical_crossentropy(
                    y_true, y_pred, n_classes=n_classes)
            loss[layer_name] = loss_function
        elif n_classes == 1:
            loss[layer_name] = MSE

    # Compile model
    model.compile(loss=loss, optimizer=optimizer)
    
elif model_type == 'pixelwise':
    model = PanopticNet(backbone,
                       input_shape=train_data.x.shape[1:],
                       norm_method='whole_image',
                       num_semantic_heads=1,
                       num_semantic_classes=[3],
                       location=False,
                       include_top=True)

    # Define loss
    loss = {}
    
    def loss_function(y_true, y_pred):
        return losses.weighted_categorical_crossentropy(
                    y_true, y_pred, n_classes=3)
    
    loss['semantic_0'] = loss_function
    
    # Compile model
    model.compile(loss=loss, optimizer=optimizer)
    
elif model_type == 'retinamask':
    model = RetinaMask(backbone,
                      num_classes=1,
                      input_shape=train_data.x.shape[1:],
                      norm_method='whole_image',
                      use_imagenet=True,
                      pyramid_levels=pyramid_levels,
                      anchor_params=anchor_params)
    
    # Define loss
    retinanet_losses = losses.RetinaNetLosses()
    loss = {
        'regression': retinanet_losses.regress_loss,
        'classification': retinanet_losses.classification_loss,
        'masks': retinanet_losses.mask_loss
    }
    
    # Compile model
    model.compile(loss=loss, optimizer=optimizer)
    
model.summary()

W0417 02:16:43.115215 139679407671104 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "panopticnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_0 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
norm (ImageNormalization2D)     (None, 128, 128, 1)  0           input_0[0][0]                    
__________________________________________________________________________________________________
location (Location2D)           (None, 128, 128, 2)  0           norm[0][0]                       
__________________________________________________________________________________________________
concatenate_location (Concatena (None, 128, 128, 3)  0           norm[0][0]                       
                                                                 location[0][0]         

## Part 4: Define model training parameters

Here, we define all of the parameters needed for training. The model trainer objects will record these metadata after training.

In [7]:
model_name = 'nuclear_{}_{}_{}_{}'.format(str(dataset_split_seed), dataset_size, backbone, model_type)
model_path = os.path.join('/data', 'models', date, model_name)
dataset_metadata={'name': dataset_name,
                  'other': 'Pooled nuclear data from HEK293, HeLa-S3, NIH-3T3, and RAW264.7 cells.'}
training_kwargs = {}
training_kwargs['batch_size'] = batch_size
training_kwargs['lr'] = 1e-5 if model_type=='retinamask' else 1e-4
training_kwargs['lr_decay'] = 0.95
training_kwargs['training_seed'] = 0
training_kwargs['n_epochs'] = n_epochs
training_kwargs['training_steps_per_epoch'] = 82800//16 if model_type == 'retinamask' else 82800 // training_kwargs['batch_size']
training_kwargs['validation_steps_per_epoch'] = val_data.y.shape[0] // training_kwargs['batch_size']

## Part 5: Create the model trainer and train the model

Here, we create the model trainer, train the model, and export the model - along with the metadata and benchmarks

In [8]:
from deepcell.model_trainer import ModelTrainer
from deepcell_toolbox import retinamask_postprocess
from deepcell_toolbox.deep_watershed import deep_watershed as watershed_postprocess 
from functools import partial
from scipy import ndimage

if model_type == 'watershed':
    postprocessing_fn = watershed_postprocess
elif model_type == 'pixelwise':
    def pixelwise(prediction, threshold=.75):
        """Post-processing for pixelwise transform predictions.
        Uses the interior predictions to uniquely label every instance.
        Args:
            prediction: pixelwise transform prediction
            threshold: confidence threshold for interior predictions
        Returns:
            post-processed data with each cell uniquely annotated
        """
        labeled = []
        for b in range(prediction.shape[0]):
            pred = prediction[b]
            interior = pred[..., 1] > threshold
            data = np.expand_dims(interior, axis=-1)
            label_image = ndimage.label(data)[0]
            labeled.append(label_image)
        labeled = np.stack(labeled, axis=0)
        return labeled
    postprocessing_fn = pixelwise
elif model_type == 'retinamask':
    postprocessing_fn = partial(retinamask_postprocess, image_shape=(128,128))
    
model_trainer = ModelTrainer(model,
                            model_name,
                            model_path,
                            train_data,
                            val_data,
                            benchmarking_data = test_dict,
                            postprocessing_fn=postprocessing_fn,
                            predict_batch_size=32,
                            dataset_metadata=dataset_metadata,
                            training_kwargs=training_kwargs)

model_trainer.create_model()

Epoch 1/16


W0417 02:17:11.179103 139679407671104 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


W0417 02:17:31.110288 139679407671104 callbacks.py:257] Method (on_train_batch_end) is slow compared to the batch update (0.161808). Check your callbacks.



Epoch 00001: val_loss improved from inf to 0.00750, saving model to /data/models/04162020/nuclear_1_128_mobilenetv2_watershed/nuclear_1_128_mobilenetv2_watershed.h5


5175/5175 - 841s - loss: 0.0074 - semantic_0_loss: 0.0033 - semantic_1_loss: 0.0033 - semantic_2_loss: 7.2259e-04 - val_loss: 0.0075 - val_semantic_0_loss: 0.0024 - val_semantic_1_loss: 0.0040 - val_semantic_2_loss: 0.0010


...

/usr/local/lib/python3.6/dist-packages/deepcell/metrics.py:112: RuntimeWarning: invalid value encountered in long_scalars
  dice = (2 * intersection.sum() / (pred.sum() + truth.sum()))
/usr/local/lib/python3.6/dist-packages/deepcell/metrics.py:113: RuntimeWarning: invalid value encountered in long_scalars
  jaccard = intersection.sum() / union.sum()
/usr/local/lib/python3.6/dist-packages/deepcell/metrics.py:114: RuntimeWarning: invalid value encountered in long_scalars
  precision = intersection.sum() / pred.sum()
/usr/local/lib/python3.6/dist-packages/deepcell/metrics.py:115: RuntimeWarning: invalid value encountered in long_scalars
  recall = intersection.sum() / truth.sum()


/usr/local/lib/python3.6/dist-packages/deepcell/metrics.py:116: RuntimeWarning: invalid value encountered in double_scalars
  Fmeasure = (2 * precision * recall) / (precision + recall)



____________Object-based statistics____________

Number of true cells:		 150490
Number of predicted cells:	 147021

Correct detections:  132814	Recall: 88.2543690610671802687647868879139423370361328125%
Incorrect detections: 14207	Precision: 90.3367546132865442132242606021463871002197265625%

Gained detections: 12876	Perc Error: 43.64154013015184574442173470743000507354736328125%
Missed detections: 15471	Perc Error: 52.43695770065075834054368897341191768646240234375%
Merges: 988		Perc Error: 3.348698481561822148933060816489160060882568359375%
Splits: 139		Perc Error: 0.471122559652928429851925784532795660197734832763671875%
Catastrophes: 30		Perc Error: 0.1016811279826464187170387276637484319508075714111328125%

Gained detections from splits: 143
Missed detections from merges: 1003
True detections involved in catastrophes: 75
Predicted detections involved in catastrophes: 61 

Average Pixel IOU (Jaccard Index): 0.83408814196867575585514487102045677602291107177734375 



/usr/local/lib/python3.6/dist-packages/deepcell/model_trainer.py:184: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  y_pred[i] = remove_small_objects(y_pred[i].astype(int), min_size=self.min_size)
/usr/local/lib/python3.6/dist-packages/deepcell/model_trainer.py:185: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  y_true[i] = remove_small_objects(y_true[i].astype(int), min_size=self.min_size)



____________Object-based statistics____________

Number of true cells:		 127382
Number of predicted cells:	 126032

Correct detections:  120602	Recall: 94.677426952002633697702549397945404052734375%
Incorrect detections: 5430	Precision: 95.6915703948203599793487228453159332275390625%

Gained detections: 4401	Perc Error: 42.62056943637420403092619380913674831390380859375%
Missed detections: 5012	Perc Error: 48.5376718961843920396859175525605678558349609375%
Merges: 802		Perc Error: 7.76680224675576180715097507345490157604217529296875%
Splits: 82		Perc Error: 0.79411195041642457237429653105209581553936004638671875%
Catastrophes: 29		Perc Error: 0.28084447026922332302234508460969664156436920166015625%

Gained detections from splits: 86
Missed detections from merges: 815
True detections involved in catastrophes: 69
Predicted detections involved in catastrophes: 59 

Average Pixel IOU (Jaccard Index): 0.8433365365855305384457096806727349758148193359375 

